In [1]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
pd.set_option('display.width', 10000)

In [2]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [3]:
copy = gg1.copy(deep=True)

In [4]:
def removePunc(x):
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\":()]+', '', x) #taking out punctuation i.e ? ! . ' and " 
    x = re.sub(r'(RT|rt) ', '', x) #taking out the initial "RT "
    x= re.sub(r'(g|G)olden (g|G)lobes*', '', x)
    return x.strip()
# copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))

In [5]:
#takes out duplicate retweets 
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])

1754153
671347


In [6]:
def findRows(x, contains, ncontains):
    s = "^"
    for word in contains:
        s += f'(?=.*{word})'
    for word in ncontains:
        s += f'(?!.*{word})'
    s += '.*$'
    if bool(re.match(s, x)):
        return True
    else:
        False
con = ['Best', 'dressed']
noncon = []
df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
ccc = df[~df['Rows'].isnull()]
print(len(ccc))

492


In [7]:
ccc.to_csv('bestdressed.csv', index=False)

In [8]:
# def findProperNouns(sen):
#     tagged_sent = pos_tag(sen.split())
#     propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
#     return propernouns
# ccc['ProperNouns'] = ccc['text'].apply(lambda x: findProperNouns(x))
# ccc.head(10)

In [9]:
def findPersonNames(text):
# text = 'Jessica Chastain so far is the best dressed Best body at the  ðŸ˜'
    names = []
    nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                if nltk_result_leaf[0] != 'Best':
                    name += nltk_result_leaf[0] + ' '
            names.append(name.strip())
#             print ('Type: ', nltk_result.label(), 'Name: ', name)
    return names

In [10]:
ccc['ProperNouns'] = ccc['text'].apply(lambda x: findPersonNames(x))

C:\Users\alanx\AppData\Local\Temp/ipykernel_20548/561391673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccc['ProperNouns'] = ccc['text'].apply(lambda x: findPersonNames(x))


In [11]:
ccc.head(10)

,text,Rows,ProperNouns
216,Best dressed so far Id say Naomi Watts Love th...,True,[Naomi Watts]
1828,OMG - Kate Beckinsdale is perfection Best dres...,True,"[OMG, Kate Beckinsdale, ]"
3252,“ SWOON ALEBest dressed men at the //” excuse...,True,[SWOON]
3617,Best dressed,True,[]
3723,Best dressed noms Rosamund Pike Jessica Chaist...,True,"[Rosamund Pike Jessica Chaistain, Jennifer Lopez]"
5268,Best dressed shut it down,True,[]
6112,GORGEOUS Best dressed in my eyes,True,"[GORGEOUS, ]"
7246,Best dressed according to me pa2 Matt Bomer Ch...,True,"[Matt Bomer Chris Pine, Eddie Redmayne]"
7556,Best dressed Julianne Moore Emma Stone Lorde T...,True,[Julianne Moore Emma Stone Lorde Taylor Schill...
7766,Best dressed couple adamlevine,True,[]


In [12]:
len(ccc)

492

In [13]:
NGRAMS_DICT = {}
for i in range(len(ccc)):
    PN = ccc['ProperNouns'].iloc[i]
    if len(PN) % 2 == 0 and len(PN) > 0: 
        for j in PN:
            if j == '': #take out empty string for some reason its added in there wack
                continue
            name = j
            if len(name.split()) > 2:
                multipleNames = name.split()
                for n in range(0, len(multipleNames)-1, 2):
                    name = multipleNames[n] + ' ' + multipleNames[n+1]
            else:
                name = j
            if name in NGRAMS_DICT:
                NGRAMS_DICT[name] += 1
            else:
                NGRAMS_DICT[name] = 1

In [14]:
cd = sorted(NGRAMS_DICT.items(),key=lambda x: (x[1], len(x[0])),reverse=True)
cd

[('Kate Hudson', 15),
 ('ALEBest', 10),
 ('SWOON', 9),
 ('Versace', 6),
 ('Kate', 6),
 ('Amal Clooney', 5),
 ('Emma Stone', 5),
 ('Awards', 5),
 ('Felicity Jones', 4),
 ('Chrissy Teigen', 4),
 ('Jessica', 4),
 ('Jennifer Lopez', 3),
 ('Julianne Moore', 3),
 ('Sienna Miller', 3),
 ('Emily Blunt', 3),
 ('Viola Davis', 3),
 ('Amy Adams', 3),
 ('OBSESSED', 3),
 ('Hudson', 3),
 ('Whos', 3),
 ('POW', 3),
 ('Amy', 3),
 ('Reese Witherspoon', 2),
 ('Allison Williams', 2),
 ('Jennifer Aniston', 2),
 ('Kerry Washington', 2),
 ('Conchita Wurst', 2),
 ('Lupita Nyongo', 2),
 ('Classic Worst', 2),
 ('Rosamund Pike', 2),
 ('John Legend', 2),
 ('Stone Naomi', 2),
 ('Danes Amal', 2),
 ('Red Carpet', 2),
 ('Jared Leto', 2),
 ('Beautiful', 2),
 ('Clooney', 2),
 ('Lupita', 2),
 ('Lanvin', 2),
 ('Simply', 2),
 ('Worst', 2),
 ('Faris', 2),
 ('Jenna', 2),
 ('Hands', 2),
 ('Lorde', 2),
 ('Emma', 2),
 ('Reem', 2),
 ('Matt', 2),
 ('Amal', 2),
 ('Per', 2),
 ('Im', 2),
 ('Giambattista Valli', 1),
 ('Supporting Act

In [15]:
text = 'Best dressed noms Rosamund Pike Jessica Chaistain &amp; Jennifer Lopez'
nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
        name = ''
        print(nltk_result)
        print(nltk_result.leaves())
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
        print ('Type: ', nltk_result.label(), 'Name: ', name)

(PERSON Rosamund/NNP Pike/NNP Jessica/NNP Chaistain/NNP)
[('Rosamund', 'NNP'), ('Pike', 'NNP'), ('Jessica', 'NNP'), ('Chaistain', 'NNP')]
Type:  PERSON Name:  Rosamund Pike Jessica Chaistain 
(PERSON Jennifer/NNP Lopez/NNP)
[('Jennifer', 'NNP'), ('Lopez', 'NNP')]
Type:  PERSON Name:  Jennifer Lopez 
